<a href="https://colab.research.google.com/github/camilacruz-cs/PretaLab/blob/main/mod6/%20aula1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preparando o ambiente

In [ ]:
# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/pasta

In [ ]:
import os
print(os.getcwd())


In [ ]:
!ls

## Cenário

**Análise de Satisfação de Clientes em uma Plataforma de E-commerce**

Vamos trabalhar com uma base de dados de satisfação de clientes retirada do Kaggle. Nosso objetivo é aplicar conceitos já estudado:

1. Manipulação de dados,
2. banco de dados,
3. Estatística descritiva,
4. probabilidade e testes de hipóteses




**Sobre o Conjunto de Dados**

***Conjunto de Dados Público de E-commerce Brasileiro da Olist***


Este é um conjunto de dados público de pedidos feitos na Olist, uma loja de e-commerce brasileira. O conjunto de dados contém informações sobre 100 mil pedidos realizados entre 2016 e 2018 em diversos marketplaces no Brasil. Suas características permitem visualizar um pedido a partir de várias dimensões: desde o status do pedido, preço, pagamento e desempenho de frete até a localização do cliente, atributos do produto e, por fim, avaliações escritas pelos clientes. Também disponibilizamos um conjunto de dados de geolocalização que relaciona os CEPs brasileiros com coordenadas de latitude/longitude.

Esses são dados comerciais reais, foram anonimizados e as referências a empresas e parceiros nos textos de avaliação foram substituídas pelos nomes das grandes casas de Game of Thrones.

Integração com o Funil de Marketing da Olist

Também liberamos um Conjunto de Dados do Funil de Marketing. Você pode unir ambos os conjuntos de dados e agora ver um pedido pela perspectiva de Marketing!

As instruções para realizar a junção estão disponíveis neste Kernel.


***Contexto***

Este conjunto de dados foi gentilmente fornecido pela Olist, a maior loja de departamentos nos marketplaces brasileiros. A Olist conecta pequenos negócios de todo o Brasil a canais de vendas de forma simples e com um único contrato. Esses comerciantes podem vender seus produtos através da Olist e enviá-los diretamente aos clientes usando parceiros logísticos da Olist. Veja mais em nosso site: www.olist.com.

Após um cliente comprar um produto na Olist, o vendedor é notificado para atender o pedido. Quando o cliente recebe o produto, ou quando a data de entrega estimada chega ao fim, o cliente recebe uma pesquisa de satisfação por e-mail, onde pode dar uma nota para a experiência de compra e deixar alguns comentários.

Atenção

Um pedido pode conter múltiplos itens.
Cada item pode ser atendido por um vendedor diferente.


## Etapa 1: Manipulação de Arquivos e Módulos em Python

In [ ]:
!pip install kaggle


In [ ]:
import os
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sqlite3
import numpy as np

Dataset  Conjunto de Dados Público de E-commerce Brasileiro da Olist

In [ ]:
import kagglehub

# Download selected version
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce/versions/7")

print("Path to dataset files:", path)

In [ ]:
# Carregando a base de clientes
customers = pd.read_csv('/root/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/7/olist_customers_dataset.csv')

# Carregando a base de pedidos
orders = pd.read_csv('/root/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/7/olist_orders_dataset.csv')

# Carregando a base de itens vendidos
order_items = pd.read_csv('/root/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/7/olist_order_items_dataset.csv')

# Carregando a base de marketing e reviews (satisfação)
marketing_reviews = pd.read_csv('/root/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/7/olist_order_reviews_dataset.csv')

reviews = pd.read_csv('/root/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/7/olist_order_reviews_dataset.csv')


In [ ]:
# Verificando valores faltantes
print(customers.isnull().sum())
print(orders.isnull().sum())
print(order_items.isnull().sum())
print(reviews.isnull().sum())

In [ ]:
# Tratamento de dados: preenchendo valores faltantes
customers.fillna('Unknown', inplace=True)
orders['order_delivered_customer_date'].fillna(method='ffill', inplace=True)

In [ ]:
# Exemplo de transformação usando Numpy - criando uma coluna de dias para entrega
orders['delivery_time_days'] = np.where(orders['order_delivered_customer_date'].notnull(),
                                        (pd.to_datetime(orders['order_delivered_customer_date']) -
                                         pd.to_datetime(orders['order_purchase_timestamp'])).dt.days,
                                        np.nan)


In [ ]:
# Verificando os dados transformados
orders.head()

Segundo dataset são vendas realizadas na plataforma.

## Passo 2: Criar e inserir dados no banco de dados SQLite

Vamos alimentar um banco de dados SQLite com os  datasets. A vantagem disso é que podemos realizar consultas complexas para responder a perguntas de negócio de forma eficiente, como entender quais campanhas geraram mais vendas ou os padrões de comportamento dos clientes.

**Criando e populando o banco de dados SQLite:**


In [ ]:
# Conectar ao banco de dados SQLite (ou criar, se não existir)
conn = sqlite3.connect('olist_marketing_sales.db')
cursor = conn.cursor()

In [ ]:
# Criar tabelas no banco de dados para armazenar os dados
customers.to_sql('customers', conn, if_exists='replace', index=False)
orders.to_sql('orders', conn, if_exists='replace', index=False)
order_items.to_sql('order_items', conn, if_exists='replace', index=False)
reviews.to_sql('reviews', conn, if_exists='replace', index=False)

In [ ]:
# Conferir se as tabelas foram criadas corretamente
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
# Commitar as mudanças e fechar a conexão
conn.commit()
conn.close()

 Esse cruzamento vai permitir entendermos, por exemplo, quais tipos de campanhas de marketing resultaram em maiores vendas ou em uma melhor retenção de clientes.

## Passo 3: Consultas SQL para cruzar os dados

Agora que temos ambos os datasets no banco de dados SQLite, podemos começar a cruzar as informações de marketing e vendas para analisar como as campanhas impactaram as vendas.

In [ ]:
# Reabrir conexão com o banco de dados
conn = sqlite3.connect('olist_marketing_sales.db')
cursor = conn.cursor()

In [ ]:
# Consultar as vendas que foram impactadas por campanhas
# Consulta SQL para cruzar vendas, clientes e avaliações
query = '''
    SELECT c.customer_id, o.order_id, oi.product_id, r.review_score, o.order_purchase_timestamp
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN reviews r ON o.order_id = r.order_id
    WHERE r.review_score IS NOT NULL;
'''

In [ ]:
# Executar a consulta e armazenar o resultado em um DataFrame
df_crossed_data = pd.read_sql_query(query, conn)

print("Dados Cruzados:")
print(df_crossed_data.head())

In [ ]:
# Fechar conexão com o banco
conn.close()

## Passo 4: Análise e visualização dos dados

Agora, com o cruzamento dos dados pronto, podemos realizar a análise estatística e as visualizações necessárias para entender melhor o impacto das campanhas nas vendas.

Análise Estatística (Testes de Hipóteses, Probabilidade e Amostragem):
Probabilidade e Distribuições:
Qual a probabilidade de uma campanha específica resultar em uma venda?
Qual a taxa de conversão por campanha?

In [ ]:
# Calcular a probabilidade de uma campanha resultar em uma venda positiva (review_score > 3)
vendas_positivas = len(df_crossed_data[df_crossed_data['review_score'] > 3])
total_vendas = len(df_crossed_data)
prob_venda = (vendas_positivas / total_vendas) * 100

print(f'Probabilidade de uma campanha resultar em venda positiva: {prob_venda:.2f}%')


**Testes de Hipóteses (ANOVA):**

Vamos usar ANOVA para verificar se existe uma diferença significativa nas avaliações entre diferentes tipos de campanhas (supondo que temos dados de tipo de campanha).

In [ ]:
from scipy import stats


In [ ]:
# Supondo que temos uma coluna 'campaign_type' na tabela de Marketing
# Aqui, apenas para fins de exemplo, adicionamos uma coluna fictícia de tipo de campanha
# Em um cenário real, essa coluna deve estar presente nos dados

# Adicionar coluna fictícia de tipo de campanha para exemplo
np.random.seed(42)
campaign_types = ['Email', 'Social Media', 'Google Ads']
df_crossed_data['campaign_type'] = np.random.choice(campaign_types, size=len(df_crossed_data))

In [ ]:
# Realizar ANOVA para comparar a média de avaliações de diferentes tipos de campanha
anova_result = stats.f_oneway(
    df_crossed_data[df_crossed_data['campaign_type'] == 'Email']['review_score'],
    df_crossed_data[df_crossed_data['campaign_type'] == 'Social Media']['review_score'],
    df_crossed_data[df_crossed_data['campaign_type'] == 'Google Ads']['review_score']
)

print(f'Resultado ANOVA: Estatística F = {anova_result.statistic:.4f}, Valor-p = {anova_result.pvalue:.4f}')



In [ ]:
# Interpretação
if anova_result.pvalue < 0.05:
    print("Rejeitamos a hipótese nula: há uma diferença significativa nas avaliações entre os tipos de campanha.")
else:
    print("Não rejeitamos a hipótese nula: não há diferença significativa nas avaliações entre os tipos de campanha.")

## Passo 5:Análise da Satisfação do Cliente para Vendas Impactadas

Vamos verificar se as vendas impactadas pelas campanhas têm avaliações mais positivas, indicando maior satisfação dos clientes.

In [ ]:
# Filtrar vendas impactadas por campanhas (review_score > 3)
vendas_impactadas = df_crossed_data[df_crossed_data['review_score'] > 3]

In [ ]:
# Calcular a média das avaliações para vendas impactadas
media_satisfacao = vendas_impactadas['review_score'].mean()
print(f'Média de satisfação dos clientes impactados por campanhas: {media_satisfacao:.2f}')

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(vendas_impactadas['review_score'], bins=5, kde=True, color='green')
plt.title('Distribuição das Avaliações - Vendas Impactadas por Campanhas')
plt.xlabel('Review Score')
plt.ylabel('Frequência')
plt.show()

## Passo 6: Cálculo da Taxa de Conversão Geral

In [ ]:
# Conectar ao banco de dados novamente
conn = sqlite3.connect('olist_marketing_sales.db')

In [ ]:
# Consultar o total de clientes
query_total_clients = 'SELECT COUNT(*) FROM customers;'
total_clientes = pd.read_sql_query(query_total_clients, conn).iloc[0,0]

In [ ]:
# Consultar o número de clientes que realizaram pelo menos uma compra
query_clientes_compras = '''
    SELECT COUNT(DISTINCT customer_id)
    FROM orders;
'''

In [ ]:
clientes_compras = pd.read_sql_query(query_clientes_compras, conn).iloc[0,0]


In [ ]:
# Calcular a taxa de conversão geral
taxa_conversao_geral = (clientes_compras / total_clientes) * 100
print(f'Taxa de Conversão Geral: {taxa_conversao_geral:.2f}%')

In [ ]:
# Fechar a conexão
conn.close()